### Introduction
In the past decade, both the NBA and EuroLeague have seen a notable increase in 3-point shooting. While it’s often assumed that shooting more 3s could lead to a drop in accuracy, the data may tell a different story. This project investigates whether teams have become not only more reliant on 3-pointers, but also more efficient, challenging assumptions about volume vs. efficiency in modern basketball.

### Objectives
- Analyze YoY trends in average 3PA and 3P%
- Evaluate the correlation between increased volume and accuracy
- Compare the league-wide shooting evolution in both competitions

### Hypothesis
Over the last 10 years, the rise in average 3-point attempts (av3PA) has not lowered 3-point percentage (3P%) in the NBA or EuroLeague, in fact, 3P% has improved year-over-year.

### Metrics involved
- av3P (Average 3 Point Attempts): The number of shots a team takes from beyond the three-point line per season on average.
- 3P% (Three-Point Percentage): Measures how accurately teams shoot from beyond the three point line.

In [27]:
# install the NBA API

!pip install nba_api

In [28]:
# install the Euroleague API

!pip install euroleague-api

In [29]:
# import numpy, pandas and requests

import pandas as pd
import numpy as np
import requests

## EUROLEAGUE DATA

In [31]:
######### Import euroleague data

from euroleague_api.team_stats import TeamStats


In [32]:
ts = TeamStats("E")

In [33]:
# test to see how the function works for a single year
df_euro1 = ts.get_team_stats_single_season("traditional", 2014)
df_euro1

,teamRanking,gamesPlayed,minutesPlayed,pointsScored,twoPointersMade,twoPointersAttempted,twoPointersPercentage,threePointersMade,threePointersAttempted,threePointersPercentage,...,turnovers,blocks,blocksAgainst,foulsCommited,foulsDrawn,pir,team.code,team.tvCodes,team.name,team.imageUrl
0,1,24.0,40.000000,71.6,19.0,39.8,47.9%,7.1,19.6,36.4%,...,14.6,3.1,3.2,22.1,20.5,73.5,ZAL,ZAL,Zalgiris Kaunas,https://media-cdn.incrowdsports.com/0aa09358-3...
1,2,24.0,40.208333,72.4,21.3,41.5,51.3%,6.0,16.8,36.1%,...,15.7,2.0,2.8,23.5,19.8,73.6,BER,ALB,ALBA Berlin,https://media-cdn.incrowdsports.com/ccc34858-2...
2,3,24.0,40.000000,77.4,20.2,39.4,51.2%,7.7,24.2,31.7%,...,10.0,2.4,3.2,23.3,20.8,76.6,MIL,EA7,EA7 Emporio Armani Milan,https://media-cdn.incrowdsports.com/8154f184-c...
3,4,28.0,40.000000,75.7,19.3,39.4,49.1%,7.5,21.6,34.8%,...,12.1,2.9,3.3,22.6,20.8,81.4,IST,EFS,Anadolu Efes Istanbul,https://media-cdn.incrowdsports.com/8ea8cec7-d...
4,5,24.0,40.416667,75.4,21.7,43.4,50%,6.0,20.2,29.5%,...,14.8,2.3,3.5,21.2,20.5,81.8,RED,CZT,Crvena Zvezda Meridianbet Belgrade,https://media-cdn.incrowdsports.com/d2eef4a8-6...
5,6,28.0,40.000000,75.1,17.9,35.3,50.6%,8.8,23.2,37.8%,...,12.7,2.3,2.8,21.4,20.8,81.8,PAN,PAO,Panathinaikos AKTOR Athens,https://media-cdn.incrowdsports.com/e3dff28a-9...
6,7,30.0,40.166667,74.3,18.2,35.1,51.8%,8.3,23.6,35.1%,...,11.6,3.3,2.5,22.4,21.5,81.9,OLY,OLY,Olympiacos Piraeus,https://media-cdn.incrowdsports.com/789423ac-3...
7,8,24.0,40.208333,78.5,20.3,41.2,49.3%,8.3,21.6,38.4%,...,12.0,3.2,2.8,22.9,19.5,82.5,NOV,NIZ,BC Nizhny Novgorod,https://media-cdn.incrowdsports.com/dddcfb2d-2...
8,9,24.0,40.625000,76.1,19.0,37.6,50.4%,7.6,22.5,33.9%,...,12.8,2.0,2.6,17.2,21.7,82.6,GAL,GSL,Galatasaray Spor Kulubu,https://media-cdn.incrowdsports.com/d8c1e9ac-0...
9,10,27.0,40.370370,77.1,22.0,41.1,53.5%,7.1,21.0,34%,...,12.9,4.4,2.5,21.1,19.8,85.6,TEL,MAC,Maccabi Playtika Tel Aviv,https://media-cdn.incrowdsports.com/5c55ef14-2...


In [34]:
df_euro1.columns

Index(['teamRanking', 'gamesPlayed', 'minutesPlayed', 'pointsScored',
       'twoPointersMade', 'twoPointersAttempted', 'twoPointersPercentage',
       'threePointersMade', 'threePointersAttempted',
       'threePointersPercentage', 'freeThrowsMade', 'freeThrowsAttempted',
       'freeThrowsPercentage', 'offensiveRebounds', 'defensiveRebounds',
       'totalRebounds', 'assists', 'steals', 'turnovers', 'blocks',
       'blocksAgainst', 'foulsCommited', 'foulsDrawn', 'pir', 'team.code',
       'team.tvCodes', 'team.name', 'team.imageUrl'],
      dtype='object')

In [187]:
# Create lists of years with my desired rangea and a list where I will compile the season dataframes
years = list(range(2014, 2025))
season_dfs = []

# loop to obtain the df of single stats per team per season, formatting for the season, filtering for the columns that I need 
# and appending it to the list I created above

for year in years:
    df = ts.get_team_stats_single_season("traditional", year)
    
    df["season"] = f"{year}-{str(year + 1)}" 
    
    df_clean = df[["threePointersAttempted", "threePointersPercentage", "team.name", "season", "twoPointersAttempted"]]
    
    season_dfs.append(df_clean)

In [189]:
# Combine all seasons into one DataFrame
df_euro_all = pd.concat(season_dfs)

# Rename columns
df_euro_all.rename(columns={
    "threePointersAttempted": "eur_3PA",
    "threePointersPercentage": "eur_3P%",
    "team.name": "team",
    "twoPointersAttempted": "eur_2PA"
}, inplace=True)

df_euro_all

,eur_3PA,eur_3P%,team,season,eur_2PA
0,19.6,36.4%,Zalgiris Kaunas,2014-2015,39.8
1,16.8,36.1%,ALBA Berlin,2014-2015,41.5
2,24.2,31.7%,EA7 Emporio Armani Milan,2014-2015,39.4
3,21.6,34.8%,Anadolu Efes Istanbul,2014-2015,39.4
4,20.2,29.5%,Crvena Zvezda Meridianbet Belgrade,2014-2015,43.4
...,...,...,...,...,...
13,22.7,36.2%,AS Monaco,2024-2025,40.9
14,24.7,38.7%,FC Barcelona,2024-2025,40.7
15,24.3,38.5%,Anadolu Efes Istanbul,2024-2025,38.3
16,25.9,37.3%,Olympiacos Piraeus,2024-2025,35.4


In [191]:
# Strip the '%' symbol and convert to float
df_euro_all['eur_3P%'] = df_euro_all['eur_3P%'].str.rstrip('%').astype('float')

In [193]:
# Create a copy for filtering
df_euro_filtered = df_euro_all.copy()

# Group by season, find the avg and restart the index
euro_avg = df_euro_filtered.groupby("season")[["eur_3PA", "eur_3P%", "eur_2PA"]].mean().reset_index()

euro_avg = euro_avg.round(2)
euro_avg

,season,eur_3PA,eur_3P%,eur_2PA
0,2014-2015,22.01,35.38,39.76
1,2015-2016,22.65,35.87,38.20
2,2016-2017,22.61,37.19,38.96
3,2017-2018,22.81,37.56,38.54
4,2018-2019,23.13,36.85,38.33
5,2019-2020,24.75,37.24,36.96
6,2020-2021,23.86,37.91,36.33
7,2021-2022,24.18,35.57,36.38
8,2022-2023,25.13,36.11,35.77
9,2023-2024,25.33,36.37,37.09


## NBA DATA

In [40]:
########### import nba data

from nba_api.stats.endpoints import leaguedashteamstats

In [41]:
nba_ts = leaguedashteamstats

In [42]:
# test to see how the function works for a single year
df_nba1 = nba_ts.LeagueDashTeamStats(
        season="2015-16",
        measure_type_detailed_defense="Base",
        per_mode_detailed="PerGame")

df_nba_data = df_nba1.get_data_frames()[0]
df_nba_data.head

<bound method NDFrame.head of        TEAM_ID               TEAM_NAME  GP   W   L  W_PCT   MIN   FGM   FGA  \
0   1610612737           Atlanta Hawks  82  48  34  0.585  48.4  38.6  84.4   
1   1610612738          Boston Celtics  82  48  34  0.585  48.2  39.2  89.2   
2   1610612751           Brooklyn Nets  82  21  61  0.256  48.2  38.2  84.4   
3   1610612766       Charlotte Hornets  82  48  34  0.585  48.4  37.0  84.4   
4   1610612741           Chicago Bulls  82  42  40  0.512  48.5  38.6  87.4   
5   1610612739     Cleveland Cavaliers  82  57  25  0.695  48.4  38.7  84.0   
6   1610612742        Dallas Mavericks  82  42  40  0.512  48.8  37.4  84.1   
7   1610612743          Denver Nuggets  82  33  49  0.402  48.4  37.7  85.4   
8   1610612765         Detroit Pistons  82  44  38  0.537  48.5  37.9  86.4   
9   1610612744   Golden State Warriors  82  73   9  0.890  48.5  42.5  87.3   
10  1610612745         Houston Rockets  82  41  41  0.500  48.4  37.7  83.5   
11  1610612754        

In [43]:
df_nba_data.columns

Index(['TEAM_ID', 'TEAM_NAME', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA',
       'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB',
       'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS',
       'PLUS_MINUS', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK',
       'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK',
       'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK',
       'DREB_RANK', 'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK',
       'BLKA_RANK', 'PF_RANK', 'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK'],
      dtype='object')

In [225]:
nba_data = []

for year in range(2014, 2025):
    season = f"{year}-{str(year + 1)[-2:]}"
    
    # API call
    stats = nba_ts.LeagueDashTeamStats(
        season=season,
        measure_type_detailed_defense="Base",
        per_mode_detailed="PerGame"
    )
    
    df_nba = stats.get_data_frames()[0]
    
    # keep only relevant columns
    df_filtered = df_nba[['TEAM_NAME', 'FG3A', 'FG3_PCT', 'FGA']].copy()

    # rename columns
    df_filtered.rename(columns={
    "TEAM_NAME": "team",
    "FG3A": "nba_3PA",
    "FG3_PCT": "nba_3P%",
    "FGA": "nba_FGA"
    }, inplace=True)

    # add a new column name "season", to match it later with the euroleague dataframe
    df_filtered['season'] = season
    
    # append it to the nba_data list
    nba_data.append(df_filtered)
    

In [227]:
nba_data

[                      team  nba_3PA  nba_3P%  nba_FGA   season
 0            Atlanta Hawks     26.2    0.380     81.7  2014-15
 1           Boston Celtics     24.6    0.327     87.9  2014-15
 2            Brooklyn Nets     19.9    0.331     83.0  2014-15
 3        Charlotte Hornets     19.1    0.318     84.5  2014-15
 4            Chicago Bulls     22.3    0.353     82.9  2014-15
 5      Cleveland Cavaliers     27.5    0.367     82.2  2014-15
 6         Dallas Mavericks     25.4    0.352     85.8  2014-15
 7           Denver Nuggets     24.8    0.325     87.3  2014-15
 8          Detroit Pistons     24.9    0.344     85.8  2014-15
 9    Golden State Warriors     27.0    0.398     87.0  2014-15
 10         Houston Rockets     32.7    0.348     83.3  2014-15
 11          Indiana Pacers     21.2    0.352     83.2  2014-15
 12    Los Angeles Clippers     26.9    0.376     83.3  2014-15
 13      Los Angeles Lakers     18.9    0.344     85.6  2014-15
 14       Memphis Grizzlies     15.2    

In [231]:
df_nba_all = pd.concat(nba_data)
df_nba_all

,team,nba_3PA,nba_3P%,nba_FGA,season
0,Atlanta Hawks,26.2,0.380,81.7,2014-15
1,Boston Celtics,24.6,0.327,87.9,2014-15
2,Brooklyn Nets,19.9,0.331,83.0,2014-15
3,Charlotte Hornets,19.1,0.318,84.5,2014-15
4,Chicago Bulls,22.3,0.353,82.9,2014-15
...,...,...,...,...,...
25,Sacramento Kings,35.2,0.357,90.1,2024-25
26,San Antonio Spurs,39.6,0.357,89.8,2024-25
27,Toronto Raptors,34.0,0.348,91.0,2024-25
28,Utah Jazz,39.8,0.350,88.7,2024-25


In [249]:
df_nba_all['nba_3P%'] = df_nba_all['nba_3P%'].astype(float)
df_nba_avg = df_nba_all.groupby('season')[['nba_3PA', 'nba_3P%','nba_FGA']].mean().reset_index()
df_nba_avg

,season,nba_3PA,nba_3P%,nba_FGA
0,2014-15,22.410000,0.349100,83.563333
1,2015-16,24.083333,0.352767,84.570000
2,2016-17,27.003333,0.357167,85.416667
3,2017-18,28.996667,0.361733,86.063333
4,2018-19,32.006667,0.355500,89.213333
5,2019-20,34.103333,0.357767,88.803333
6,2020-21,34.636667,0.366033,88.416667
7,2021-22,35.180000,0.353567,88.090000
8,2022-23,34.206667,0.360133,88.316667
9,2023-24,35.106667,0.365667,88.903333


### COMPARING DATAFRAMES AND STANDARDIZING VALUES/COLUMNS

In [49]:
# Standardizing "season" column

In [141]:
df_nba_all

,team,nba_3PA,nba_3P%,season
0,Atlanta Hawks,26.2,38.0,2014-15
1,Boston Celtics,24.6,32.7,2014-15
2,Brooklyn Nets,19.9,33.1,2014-15
3,Charlotte Hornets,19.1,31.8,2014-15
4,Chicago Bulls,22.3,35.3,2014-15
...,...,...,...,...
25,Sacramento Kings,35.2,35.7,2024-25
26,San Antonio Spurs,39.6,35.7,2024-25
27,Toronto Raptors,34.0,34.8,2024-25
28,Utah Jazz,39.8,35.0,2024-25


In [83]:
df_nba_all['season'].dtype

dtype('O')

In [103]:
def format_season(season):
    start_year = int(season[:4])
    end_year = start_year + 1
    return f"{start_year}-{end_year}"

In [251]:
df_nba_avg['season'] = df_nba_avg['season'].apply(format_season)
df_nba_avg

,season,nba_3PA,nba_3P%,nba_FGA
0,2014-2015,22.410000,0.349100,83.563333
1,2015-2016,24.083333,0.352767,84.570000
2,2016-2017,27.003333,0.357167,85.416667
3,2017-2018,28.996667,0.361733,86.063333
4,2018-2019,32.006667,0.355500,89.213333
5,2019-2020,34.103333,0.357767,88.803333
6,2020-2021,34.636667,0.366033,88.416667
7,2021-2022,35.180000,0.353567,88.090000
8,2022-2023,34.206667,0.360133,88.316667
9,2023-2024,35.106667,0.365667,88.903333


In [237]:
# Standardizing "nba_3P%" column

In [253]:
df_nba_avg['nba_3P%'] = df_nba_avg['nba_3P%'] * 100

In [259]:
df_nba_avg

,season,nba_3PA,nba_3P%,nba_FGA
0,2014-2015,22.41,34.91,83.56
1,2015-2016,24.08,35.28,84.57
2,2016-2017,27.00,35.72,85.42
3,2017-2018,29.00,36.17,86.06
4,2018-2019,32.01,35.55,89.21
5,2019-2020,34.10,35.78,88.80
6,2020-2021,34.64,36.60,88.42
7,2021-2022,35.18,35.36,88.09
8,2022-2023,34.21,36.01,88.32
9,2023-2024,35.11,36.57,88.90


In [275]:
df_nba_avg = df_nba_avg.round(2)

### Final two dataframes

In [261]:
df_nba_avg

,season,nba_3PA,nba_3P%,nba_FGA
0,2014-2015,22.41,34.91,83.56
1,2015-2016,24.08,35.28,84.57
2,2016-2017,27.00,35.72,85.42
3,2017-2018,29.00,36.17,86.06
4,2018-2019,32.01,35.55,89.21
5,2019-2020,34.10,35.78,88.80
6,2020-2021,34.64,36.60,88.42
7,2021-2022,35.18,35.36,88.09
8,2022-2023,34.21,36.01,88.32
9,2023-2024,35.11,36.57,88.90


In [199]:
euro_avg

,season,eur_3PA,eur_3P%,eur_2PA
0,2014-2015,22.01,35.38,39.76
1,2015-2016,22.65,35.87,38.20
2,2016-2017,22.61,37.19,38.96
3,2017-2018,22.81,37.56,38.54
4,2018-2019,23.13,36.85,38.33
5,2019-2020,24.75,37.24,36.96
6,2020-2021,23.86,37.91,36.33
7,2021-2022,24.18,35.57,36.38
8,2022-2023,25.13,36.11,35.77
9,2023-2024,25.33,36.37,37.09


### Extra columns: 
- NBA (FGA to compare to 3PA and get fraction)
- EuroLeague (2PA to add them to 3PA, obtain FGA, and then compare them to 3PA and get fraction).

In [203]:
# I modified the previous functions to surface FGA in the NBA and 2PA in the Euroleague.
# I need to add a new column to each DataFrame that shows the proportion of 3-point attempts (3PA) out of total field goal attempts (FGA).
# I call it 3PA_ratio

In [271]:
df_nba_avg['nba_3PA_ratio'] = df_nba_avg['nba_3PA'] / df_nba_avg["nba_FGA"] *100

In [279]:
df_nba_avg.drop(columns=['3PA_ratio'], inplace=True)

In [283]:
df_nba_avg # this dataset is ready to be merged

,season,nba_3PA,nba_3P%,nba_FGA,nba_3PA_ratio
0,2014-2015,22.41,34.91,83.56,26.82
1,2015-2016,24.08,35.28,84.57,28.47
2,2016-2017,27.00,35.72,85.42,31.61
3,2017-2018,29.00,36.17,86.06,33.70
4,2018-2019,32.01,35.55,89.21,35.88
5,2019-2020,34.10,35.78,88.80,38.40
6,2020-2021,34.64,36.60,88.42,39.18
7,2021-2022,35.18,35.36,88.09,39.94
8,2022-2023,34.21,36.01,88.32,38.73
9,2023-2024,35.11,36.57,88.90,39.49


In [303]:
# For the Euroleague I need to create a new column that adds 2PA and 3PA, and name it as eur_FGA.
# Then I need to create a new column that comapres FGA to 3PA to get the ratio, similar to what I did with the nba dataframe.
# Then I need to drop the 2PA column

euro_avg

,season,eur_3PA,eur_3P%,eur_FGA,eur_3PA_ratio
0,2014-2015,22.01,35.38,61.77,35.63
1,2015-2016,22.65,35.87,60.85,37.22
2,2016-2017,22.61,37.19,61.57,36.72
3,2017-2018,22.81,37.56,61.35,37.18
4,2018-2019,23.13,36.85,61.46,37.63
5,2019-2020,24.75,37.24,61.71,40.11
6,2020-2021,23.86,37.91,60.19,39.64
7,2021-2022,24.18,35.57,60.56,39.93
8,2022-2023,25.13,36.11,60.90,41.26
9,2023-2024,25.33,36.37,62.42,40.58


In [287]:
euro_avg['eur_FGA'] = euro_avg['eur_3PA'] + euro_avg['eur_2PA']

In [291]:
euro_avg['eur_3PA_ratio'] = euro_avg['eur_3PA'] / euro_avg['eur_FGA'] *100

In [301]:
euro_avg.drop(columns=['eur_2PA'], inplace=True)

In [299]:
euro_avg = euro_avg.round(2)

In [305]:
# Merging the two dataframes

merged_df = pd.merge(df_nba_avg, euro_avg, on='season')
merged_df

,season,nba_3PA,nba_3P%,nba_FGA,nba_3PA_ratio,eur_3PA,eur_3P%,eur_FGA,eur_3PA_ratio
0,2014-2015,22.41,34.91,83.56,26.82,22.01,35.38,61.77,35.63
1,2015-2016,24.08,35.28,84.57,28.47,22.65,35.87,60.85,37.22
2,2016-2017,27.00,35.72,85.42,31.61,22.61,37.19,61.57,36.72
3,2017-2018,29.00,36.17,86.06,33.70,22.81,37.56,61.35,37.18
4,2018-2019,32.01,35.55,89.21,35.88,23.13,36.85,61.46,37.63
5,2019-2020,34.10,35.78,88.80,38.40,24.75,37.24,61.71,40.11
6,2020-2021,34.64,36.60,88.42,39.18,23.86,37.91,60.19,39.64
7,2021-2022,35.18,35.36,88.09,39.94,24.18,35.57,60.56,39.93
8,2022-2023,34.21,36.01,88.32,38.73,25.13,36.11,60.90,41.26
9,2023-2024,35.11,36.57,88.90,39.49,25.33,36.37,62.42,40.58


In [307]:
merged_df.to_csv('cleaned_dataframe', index=False)